In [6]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 23 11:07:57 2019

@author: anama
"""

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import Dense,Activation, Dropout
from keras.optimizers import Adam

train_data_dir = "fruits-360/Test"
test_data_dir = "fruits-360/Training"
validation_data_dir = "fruits-360/Validation"

batch_size = 32
nb_epochs = 20
img_height = 30
img_width = 30


#TO-DO: USE ALREADY IMPLEMENTED MODELS
#
#


train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = test_datagen.flow_from_directory(
     train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)



Found 14323 images belonging to 103 classes.
Found 0 images belonging to 103 classes.
Instructions for updating:
Use tf.cast instead.


ValueError: `validation_steps=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `validation_steps` or use the `keras.utils.Sequence` class.